## Web Scraping: Cotización Cedears Argentina
---
https://iol.invertironline.com/mercado/cotizaciones/argentina/cedears/todos

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import date
import pymysql

### Extracción primaria de Datos:
---

In [2]:
# Página principal:
url = 'https://iol.invertironline.com/mercado/cotizaciones/argentina/cedears/todos'

# Realizo el request:
r = requests.get(url)

# Chequeo de status:
r.status_code #200

200

In [3]:
# Creamos la 'sopa':
s = BeautifulSoup(r.text, 'lxml')

In [4]:
# Separamos la tabla de datos:
tabla = s.find('tbody')

In [20]:
# Extraemos los nombres de los CEDEAR:
nombres = tabla.find_all('b')
cedear = [nombre.get_text(strip=True) for nombre in nombres]
#print(cedear)
len(cedear)

345

In [9]:
# Extraemos los nombres de las Empresas:
links_nombres = tabla.find_all('a')
empresa = [link.get('title') for link in links_nombres]
#print(empresa)
len(empresa)

345

In [31]:
# Extraemos los valores de Apertura:
val_apertura = tabla.find_all('td', {"data-field": "Apertura"})
apertura = [valor.get_text(strip=True) for valor in val_apertura]
#print(apertura)
len(apertura)

345

In [32]:
# Extraemos los valores Mínimos:
val_minimo = tabla.find_all('td', {"data-field": "Minimo"})
minimo = [valor.get_text(strip=True) for valor in val_minimo]
#print(minimo)
len(minimo)

345

In [33]:
# Extraemos los valores Máximos:
val_maximo = tabla.find_all('td', {"data-field": "Maximo"})
maximo = [valor.get_text(strip=True) for valor in val_maximo]
#print(maximo)
len(maximo)

345

In [34]:
# Extraemos los valores de Último Cierre:
val_cierre = tabla.find_all('td', {"data-field": "UltimoCierre"})
cierre = [valor.get_text(strip=True) for valor in val_cierre]
#print(cierre)
len(cierre)

345

In [36]:
# Extraemos los valores de Monto Operado:
registros = tabla.find_all('tr', {'data-cantidad': '1'})
monto_operado = [registro.find_all('td', class_="tar")[-1].get_text(strip=True) for registro in registros]
#print(monto_operado)
len(monto_operado)

345

### Funciones Scraper:
---

#### FUNCION extraer_tabla():

Esta función se encarga de extraer la tabla principal desde la url indicada:

In [2]:
def extraer_tabla():
    url = 'https://iol.invertironline.com/mercado/cotizaciones/argentina/cedears/todos'
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'lxml')
    tabla = s.find('tbody')
    return tabla

#### FUNCION datos_cedear_empresas():

Esta función se encarga de descargar e identificar los nombres de los CEDEAR con sus Empresas asociadas. Una misma empresa puede estar asociada a más de un CEDEAR. Esta lista inicial de Empresas contiene inconsistencias en los nombres de las mismas: Mayúsculas, falta de puntos de abreviación o espacios en blanco añadidos. Esto hace que una misma Empresa figure como dos entidades comerciales diferentes. Debemos primero corregirlas sin modificar la cantidad de registros ni el orden de los mismos.

**Inconsistencias halladas:**

* 'AbbVie Inc.', 'Abbvie Inc.'
* 'Amazon ', 'Amazon'
* 'Bioceres Crop Solutions Corp.', 'Bioceres Crop Solutions Corp'
* 'DocuSign Inc.', 'Docusign Inc'
* 'Etsy Inc.', 'Etsy Inc',
* 'Harmony Gold ', 'Harmony Gold'
* 'Ibm', 'IBM'
* 'Jpmorgan Chase & Co.', 'JPMorgan Chase & Co.'
* 'Mastercard Inc.', 'Mastercard'
* "McDonald's", "Mcdonald's"
* 'Novartis Ag', 'Novartis AG'
* 'Snowflake Inc.', 'Snowflake Inc'
* 'Square Inc.', 'Square Inc'
* 'At&T', 'AT&T'
* 'Zoom Video Communications Inc.', 'Zoom Video Communications Inc'

In [3]:
def datos_cedear_empresas():
    # Extraemos la tabla:
    tabla = extraer_tabla()
    
    # Extraemos los nombres de los CEDEAR:
    nombres = tabla.find_all('b')
    cedear = [nombre.get_text(strip=True) for nombre in nombres]
    
    # Extraemos los nombres de las Empresas:
    links_nombres = tabla.find_all('a')
    empresa = [link.get('title') for link in links_nombres]

    # Las asociamos en el orden de extracción y creamos el DataFrame:
    dic = {"CEDEAR": cedear, "EMPRESA": empresa}
    df = pd.DataFrame(dic)

    # Corregimos inconsistencias en los nombres:
    erroneo = ['Abbvie Inc.', 'Amazon ', 'Bioceres Crop Solutions Corp', 'Docusign Inc', 'Etsy Inc', 'Harmony Gold ', 'Ibm',
               'Jpmorgan Chase & Co.', 'Mastercard', "Mcdonald's", 'Novartis Ag', 'Snowflake Inc', 'Square Inc', 'At&T',
               'Zoom Video Communications Inc']
    correcto = ['AbbVie Inc.', 'Amazon', 'Bioceres Crop Solutions Corp.', 'DocuSign Inc.', 'Etsy Inc.', 'Harmony Gold', 'IBM',
                'JPMorgan Chase & Co.', 'Mastercard Inc.', "McDonald's", 'Novartis AG', 'Snowflake Inc.', 'Square Inc.',
                'AT&T','Zoom Video Communications Inc.']

    for i in range(len(erroneo)):
        df.replace(erroneo[i], correcto[i], inplace=True)

    # Generamos el 'Indice de Empresa', para reconocer la pertenencia de cada CEDEAR:
    empresa_unique = df['EMPRESA'].unique()
    indices_empresa = [(list(empresa_unique).index(i) + 1) for i in df['EMPRESA']]
    df['INDICE_EMPRESA'] = indices_empresa
    return df

#### FUNCIÓN registros_diarios_cedear():

Esta función se encarga de extraer los regístros de movimiento diario pertenecientes a cada CEDEAR:

In [4]:
# Función Auxiliar 'a_float()': Convierte los valores String con puntos (separadores de miles) y comas (decimales) a float:
def a_float(lista):
    lista_corregida = []
    for i in range(len(lista)):
        # Elimino los puntos separadores de miles:
        item = lista[i].replace('.', '')
        # Reemplazo la coma decimal, por un punto:
        item = item.replace(',', '.')
        lista_corregida.append(float(item))
    return lista_corregida

def registros_diarios_cedear():
    # Extraemos la tabla:
    tabla = extraer_tabla()
    df = pd.DataFrame()
    
    # Extraemos los valores de Apertura:
    val_apertura = tabla.find_all('td', {"data-field": "Apertura"})
    apertura = [valor.get_text(strip=True) for valor in val_apertura]
    apertura = a_float(apertura)
    
    # Extraemos los valores Mínimos:
    val_minimo = tabla.find_all('td', {"data-field": "Minimo"})
    minimo = [valor.get_text(strip=True) for valor in val_minimo]
    minimo = a_float(minimo)
    
    # Extraemos los valores Máximos:
    val_maximo = tabla.find_all('td', {"data-field": "Maximo"})
    maximo = [valor.get_text(strip=True) for valor in val_maximo]
    maximo = a_float(maximo)
    
    # Extraemos los valores de Último Cierre:
    val_cierre = tabla.find_all('td', {"data-field": "UltimoCierre"})
    cierre = [valor.get_text(strip=True) for valor in val_cierre]
    cierre = a_float(cierre)
    
    # Extraemos los valores de Monto Operado:
    registros = tabla.find_all('tr', {'data-cantidad': '1'})
    monto_operado = [registro.find_all('td', class_="tar")[-1].get_text(strip=True) for registro in registros]
    monto_operado = a_float(monto_operado)
    
    # Creamos el DataFrame:
    df['APERTURA'] = apertura
    df['MINIMO'] = minimo
    df['MAXIMO'] = maximo
    df['CIERRE'] = cierre
    df['MONTO_OPERADO'] = monto_operado
    df['FECHA'] = str(date.today())
    return df

In [7]:
# Probamos las funciones de forma simultánea, generando un DataFrame común:
#df = pd.concat([datos_cedear_empresas(), registros_diarios_cedear()], axis=1)
#df

### Conexión y Guardado en BBDD:
---

La BBDD a utilizar constará de 3 tablas:

   * Tabla **'empresas'**: Campos: id (*primary_key, auto_increment*), nombre.
   * Tabla **'cedear'**: Campos: id (*primary_key, auto_increment*), nombre, empresa_id (*foreign_key*).
   * Tabla **'registros'**: Campos: id (*primary_key, auto_increment*), apertura, minimo, maximo, cierre, monto_operado, fecha, cedear_id (*foreign_key*).

In [5]:
# Creamos la conexión:
connection = pymysql.connect(
    host = "127.0.0.1",
    port = 3307,
    user = "root",
    password = "",
    db = "cotizaciones_pi2023"
)

# Instanciamos el cursor que se encargará de interactuar con la BBDD:
cursor = connection.cursor()

#### Tabla EMPRESAS:

In [6]:
# Extracción de datos de CEDEAR y Empresas asociadas:
df_cedear_empresas = datos_cedear_empresas()
df_cedear_empresas

,CEDEAR,EMPRESA,INDICE_EMPRESA
0,AAL,AMERICAN AIRLINES GROUP INC.,1
1,AAPL,Apple,2
2,AAPLC,Apple,2
3,AAPLD,Apple,2
4,ABBV,AbbVie Inc.,3
...,...,...,...
340,XP,XP INC.,227
341,XROX,Xerox,228
342,YELP,Yelp Inc,229
343,ZM,Zoom Video Communications Inc.,230


In [11]:
# Creación de Tabla 'empresas':
comando = "CREATE TABLE empresas (id_empresa INT NOT NULL AUTO_INCREMENT, nombre VARCHAR(100) NOT NULL, PRIMARY KEY (id_empresa));"
cursor.execute(comando)
connection.commit()

In [20]:
empresas = list(df_cedear_empresas['EMPRESA'].unique())
empresas[0]
#for i in range(len(empresas)):
#    comando = "INSERT INTO empresas (nombre) VALUES ("+empresas[i]+");"
#    cursor.execute(comando)
#connection.commit()

'AMERICAN AIRLINES GROUP INC.'